In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# Initialize the Chrome webdriver
options = Options()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

In [ ]:
def download_images(base_url, folder_path, total_pages=1):
    # Create the folder if it doesn't exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    img_count = 1

    # Initialize the Chrome webdriver
    options = Options()
    options.add_argument("--headless")  # Run Chrome in headless mode
    driver = webdriver.Chrome(options=options)

    try:
        for page in range(1, total_pages + 1):
            # Construct the URL for the current page
            url = f"{base_url}&page={page}"
            print(f"Scraping page: {page}")

            # Load the page
            driver.get(url)
            time.sleep(2)  # Add a delay to allow time for the page to load dynamically

            # Scroll down the page to load all images
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                # Scroll down to bottom
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait to load page
                time.sleep(2)

                # Calculate new scroll height and compare with last scroll height
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Extract the HTML content after all images have been loaded
            html_content = driver.page_source

            # Parse the HTML content
            soup = BeautifulSoup(html_content, 'html.parser')

            # Find the div with class 'mui-1kkefsa-gridContainer-root'
            grid_container = soup.find('div', class_='mui-1kkefsa-gridContainer-root')
            if not grid_container:
                print(f"No images found on page {page}")
                continue

            # Find all divs with the specified data-automation attribute
            divs = grid_container.find_all('div', {'data-automation': 'AssetGrids_GridItemContainer_div'})
            if not divs:
                print(f"No images found on page {page}")
                continue

            # Download and save each image
            for div in divs:
                picture = div.find('picture')
                if picture:
                    img_tag = picture.find('img')
                    if img_tag and img_tag['src']:
                        img_url = img_tag['src']
                        try:
                            img_data = requests.get(img_url).content
                            img_name = f"img{img_count}.jpg"
                            img_path = os.path.join(folder_path, img_name)
                            with open(img_path, 'wb') as f:
                                f.write(img_data)
                            print(f"Downloaded {img_name} from {img_url}")
                            img_count += 1
                        except Exception as e:
                            print(f"Failed to download image from {img_url}: {e}")
    finally:
        # Close the webdriver
        driver.quit()

In [ ]:
# # Base URL of the website with the images (excluding the page parameter)
# base_url = "https://www.shutterstock.com/search/icu-doctor-patient?image_type=photo&mreleased=true&orientation=horizontal"
# # Folder path to save the images
# folder_path = "DoctorImages"

# # Total number of pages to scrape
# total_pages = 11

# # Call the function to download images
# download_images(base_url, folder_path, total_pages)

In [ ]:
# Base URL of the website with the images (excluding the page parameter)
base_url = "https://www.shutterstock.com/search/doctor-and-patient-in-icu?mreleased=true"
# Folder path to save the images
folder_path = "DoctorImages"

# Total number of pages to scrape
total_pages = 13

# Call the function to download images
download_images(base_url, folder_path, total_pages)

In [ ]:
# downloading labelImg

# !pip3 install labelImg
import labelImg
!labelImg

In [ ]:
import json
import os

def convert_to_yolo_format(json_dir, images_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    for json_file in os.listdir(json_dir):
        # if json_file.endswith('.json'):
        with open(os.path.join(json_dir, json_file)) as f:
            data = json.load(f)
        
        image_filename = data['task']['data']['image'].split('=')[1].replace('%5C', '/')
        print(image_filename)
        image_path = os.path.join(image_filename)
        if not os.path.exists(image_path):
            print(f"Image {image_path} not found, skipping.")
            continue

        if 'result' not in data or not data['result']:
            print(f"image {image_path} has no annotations, skipping")
            continue

        width = data['result'][0]['original_width']
        height = data['result'][0]['original_height']

        yolo_annotations = []
        for annotation in data['result']:
            label = annotation['value']['rectanglelabels'][0]
            class_id = 0 if label == 'doctor' else 1  # Assuming 'doctor' is 0 and 'patient' is 1
            x_center = (annotation['value']['x'] + annotation['value']['width'] / 2) / 100
            y_center = (annotation['value']['y'] + annotation['value']['height'] / 2) / 100
            w = annotation['value']['width'] / 100
            h = annotation['value']['height'] / 100

            yolo_annotations.append(f"{class_id} {x_center} {y_center} {w} {h}")

        image_filename = data['task']['data']['image'].split('=')[1].replace('/', '%5C')
        yolo_file_path = output_dir + "/" + f"{os.path.splitext(image_filename)[0]}.txt"
        # yolo_file_path = os.path.join(output_dir, f"{os.path.splitext(image_filename)[0]}.txt")
        print(yolo_file_path)
        with open(yolo_file_path, 'w') as f:
            f.write("\n".join(yolo_annotations))
        
        # Copy image to output directory
        os.system(f"cp {image_path} {output_dir}")

json_dir = 'LabeledDoctor'
images_dir = 'DoctorImages'
output_dir = 'AnnotationsYolo'

convert_to_yolo_format(json_dir, images_dir, output_dir)


In [ ]:
import os
import shutil
import random
import json

def split_dataset(json_dir, image_dir, output_dir, split_ratio=0.8):
    os.makedirs(os.path.join(output_dir, 'train', 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'train', 'labels'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'test', 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'test', 'labels'), exist_ok=True)
    
    json_files = [f for f in os.listdir(json_dir)]
    # json_files = json_files.sort()
    print(json_files)
    random.shuffle(json_files)
    # print(len(json_files))
    
    split_point = int(len(json_files) * split_ratio)
    train_files = json_files[:split_point]
    test_files = json_files[split_point:]
    
    return train_files, test_files

json_dir = 'LabeledDoctor'
image_dir = 'DoctorImages'
output_dir = 'New_doctor_images'

train_files, test_files = split_dataset(json_dir, image_dir, output_dir)
print(train_files)
print(test_files)

In [ ]:
import os
import json

def convert_to_yolo_format(json_files, json_dir, image_dir, output_dir):
    for json_file in json_files:
        with open(os.path.join(json_dir, json_file)) as f:
            data = json.load(f)
        
        image_filename = data['task']['data']['image'].split('=')[1]
        image_path = os.path.join(image_filename.replace('%5C', '/'))
        if not os.path.exists(image_path):
            print(f"Image {image_path} not found, skipping.")
            continue

        if 'result' not in data or not data['result']:
            print(f"Image {image_path} has no annotations, skipping.")
            continue

        width = data['result'][0]['original_width']
        height = data['result'][0]['original_height']

        yolo_annotations = []
        for annotation in data['result']:
            label = annotation['value']['rectanglelabels'][0]
            class_id = 0 if label == 'doctor' else 1  # Assuming 'doctor' is 0 and 'patient' is 1
            x_center = (annotation['value']['x'] + annotation['value']['width'] / 2) / 100
            y_center = (annotation['value']['y'] + annotation['value']['height'] / 2) / 100
            w = annotation['value']['width'] / 100
            h = annotation['value']['height'] / 100

            yolo_annotations.append(f"{class_id} {x_center} {y_center} {w} {h}")

        image_filename = data['task']['data']['image'].split('=')[1].replace('/', '%5C')
        yolo_file_path = os.path.join(output_dir, 'labels', f"{os.path.splitext(image_filename)[0]}.txt")
        with open(yolo_file_path, 'w') as f:
            f.write("\n".join(yolo_annotations))
        
        # Copy image to output directory
        shutil.copy(image_path, os.path.join(output_dir, 'images', image_filename))


json_dir = 'LabeledDoctor'
image_dir = 'DoctorImages'
output_dir = 'New_doctor_images'

# Process training data
train_output_dir = os.path.join(output_dir, 'train')
convert_to_yolo_format(train_files, json_dir, image_dir, train_output_dir)

# Process testing data
test_output_dir = os.path.join(output_dir, 'test')
convert_to_yolo_format(test_files, json_dir, image_dir, test_output_dir)

In [ ]:
import os

data_path = 'New_doctor_images/data.yaml'

# Check if the data file exists
if os.path.exists(data_path):
    print(f"{data_path} exists.")
else:
    print(f"{data_path} does not exist.")

# Check if train and val directories exist
train_dir = 'New_doctor_images/train/images'
val_dir = 'New_doctor_images/test/images'

if os.path.exists(train_dir) and os.path.exists(val_dir):
    print("Train and validation directories exist.")
else:
    print("One or both directories do not exist.")

In [ ]:
from ultralytics import YOLO
import os

# Define paths
data_path = 'C:/Mini Project/Intel Project/datasets/New_doctor_images/data.yaml'
print(f"Data path: {os.path.abspath(data_path)}")  # Print absolute path to data.yaml

# Create a YOLOv8 model instance
model = YOLO('yolov8n.pt')  # You can change 'n' to 's', 'm', 'l', or 'x' for larger models

# Train the model
model.train(data=data_path, epochs=50, imgsz=640, batch=8, name='yolov8_doctor_patient14')

# Save the model
model.save('yolov8_doctor_patient.pt')

In [ ]:
import os
print("Current Working Directory:", os.getcwd())

In [ ]:
from ultralytics import YOLO
import os

# Load the trained model
model = YOLO('C:/Mini Project/Intel Project/runs/detect/yolov8_doctor_patient142/weights/best.pt')

# Directory containing test images
test_images_dir = 'C:/Mini Project/Intel Project/datasets/New_doctor_images/test/images'

# List all test images
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith(('.jpg', '.jpeg', '.png'))]

# Run predictions
results = model.predict(source=test_images, conf=0.25, save=True, save_dir='predictions')

# Display the results
for result in results:
    print(result)  # This will print the details of each prediction
    result.show()  # This will display the image with predictions (requires GUI environment)

# Evaluate the model on the validation dataset
val_results = model.val()

# Print evaluation results
print(val_results)


In [ ]:
from ultralytics import YOLO
import os

# Load the trained model
model = YOLO('C:/Mini Project/Intel Project/runs/detect/yolov8_doctor_patient142/weights/best.pt')

# Directory containing test images
test_images_dir = 'C:/Mini Project/Intel Project/ICU season 1/ep1'

# List all test images
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith(('.jpg', '.jpeg', '.png'))]

# Run predictions
results = model.predict(source=test_images, conf=0.25, save=True, save_dir='ICU_predict1')

# Display the results
for result in results:
    print(result)  # This will print the details of each prediction
    result.show()  # This will display the image with predictions (requires GUI environment)

# Evaluate the model on the validation dataset
val_results = model.val()

# Print evaluation results
print(val_results)


In [3]:
# renaming all the files of icu folder
# Function to rename multiple files
import os

def renaming():   
    folder = "ICU season 1"
    for count, filename in enumerate(os.listdir(folder)):
        dst = f"icu {str(count)}.jpg"
        src =f"{folder}/{filename}"  # foldername/filename, if .py file is outside folder
        dst =f"{folder}/{dst}"
         
        # rename() function will
        # rename all the files
        os.rename(src, dst)

renaming()